# **Исследование данных сервиса “Яндекс.Музыка” — сравнение пользователей двух городов**


**Описание проекта**

Задача проекта- На реальных данных Яндекс.Музыки c помощью библиотеки Pandas и её возможностей проверить данные и сравнить поведение и предпочтения пользователей двух столиц — Москвы и Санкт-Петербурга.

**Гипотезы**
* Активность пользователей в Москве и Санкт-Петербурге зависит от дня недели и для каждого города это проявляется по-разному; 
* Утром в понедельник и в пятницу вечером в Москве преобладают одни жанры музыки, а в Петербурге — другие;
* Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Структура проекта:**
- [Обзор данных;](#review)
- [Предобработка данных;](#preprocessing)
- [Исследовательский анализ данных.](#analysis)
- [Итоги исследования.](#results)

# Обзор данных <a id='review'></a>

#### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np

#### Просмотр данных

In [2]:
df=pd.read_csv('yandex_music_project.csv')
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


#### Информация о файле

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


#### Подсчет уникальных значений в каждом столбце

In [4]:
quantity=[]
for col in df:
    quantity.append(np.count_nonzero (df[col].unique()))
data=[
    quantity
]
columns=df.columns
table_one=pd.DataFrame(data=data,columns=columns)
table_one

,userID,Track,artist,genre,City,time,Day
0,41748,47246,43606,290,2,20392,3


### Выводы

- таблица состоит из `65077` строк, тип данных в каждом столбце: `object`
- Каждая строка таблицы содержит данные об прослушиваемом треке. Колонки описывают саму композицию (`название трека`, `исполнитель`, `жанр`), `город`, в котором находился пользователь, `время прослушивания` и `день недели`; 
- Суммарное количество пользователей, данные о которых представлены в исследовании: `41748`;
- Информация о прослушивании предоставлена только за три дня недели: `понедельник`, `среду` и `пятницу`;
- В колонках `Track`, `artist` и `genre` присутствуют пропуски данных. Необходимо устранить пропуски и выяснить не повлияют ли отсутствующие значения на результаты исследования. 


# Предобработка данных <a id='preprocessing'></a>

#### Исправление стиля заголовков столбцов

In [5]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

*Приведем наименования заголовков к единому стилю, используя "змеиный регистр" и прописные буквы:*

In [6]:
df=df.rename(columns={'  userID':'user_id','Track':'track','  City  ':'city','Day':'day'})
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

#### Исключение пропусков в данных

*Проверим наличие пропусков в каждом столбце*

In [7]:
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

*Пропуски в столбцах `track` и `artist` не влияют на результаты исследования, заменим их явными обозначениями `unknown`.
Наличие пропусков в столбце `genre` может исказить результаты исследования. Так как восстановить значения и установить причину пропусков в данном проекте невозможно заменим их явными обозначениями и оценим, насколько они повлияют на расчёты.* 



In [8]:
columns_to_replace=['track', 'artist','genre']
for column in columns_to_replace:
    df[column]=df[column].fillna('unknown')
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

#### Проверка данных на дубликаты

*Посчитаем количество явных дубликатов в таблице*

In [9]:
duplicate=df.duplicated().sum()
print('В исходных данных обнаружено',duplicate,'дубликатов')

В исходных данных обнаружено 3826 дубликатов


*Удалим все явные дубликаты из таблицы*

In [10]:
df=df.drop_duplicates()
print('Явных дубликатов в таблице:',df.duplicated().sum())

Явных дубликатов в таблице: 0


 *Для дальнейшего исследования проверим наличие неявных дубликатов в столбце с названием жанров (`genre`)*

In [11]:
list_one = df['genre'].sort_values().unique()
list_one


array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

*заменим значение `электроника` на `electronic`*

In [12]:
df['genre']=df['genre'].replace('электроника','electronic')

In [13]:
#для наглядности выведем на экран список жанров разбитый по алфавиту 
alphabet = [chr(i) for i in range(97, 123)]
list_one=df['genre'].sort_values().unique().tolist()

for i in alphabet:
    print(i, ':',*filter(lambda x: x.startswith(i), list_one),'\n')

a : acid acoustic action adult africa afrikaans alternative alternativepunk ambient americana animated anime arabesk arabic arena argentinetango art audiobook author avantgarde axé 

b : baile balkan beats bigroom black bluegrass blues bollywood bossa brazilian breakbeat breaks broadway 

c : cantautori cantopop canzone caribbean caucasian celtic chamber chanson children chill chinese choral christian christmas classical classicmetal club colombian comedy conjazz contemporary country cuban 

d : dance dancehall dancepop dark death deep deutschrock deutschspr dirty disco dnb documentary downbeat downtempo drum dub dubstep 

e : eastern easy electronic electropop emo entehno epicmetal estrada ethnic eurofolk european experimental extrememetal 

f : fado fairytail film fitness flamenco folk folklore folkmetal folkrock folktronica forró frankreich französisch french funk future 

g : gangsta garage german ghazal gitarre glitch gospel gothic grime grunge gypsy 

h : handsup hard'n'heavy har

*Просмотрим список и найдем неявные дубликаты названий жанров. Это могут быть названия с ошибками или альтернативные названия того же жанра. 
В данном датасете обнаружены неявные дубликаты жанра `hiphop`:*
* `hip` 
* `hip-hop` 
* `hop`
  
Заменим каждое значение из списка дубликатов на верное:


In [14]:
drop_genre_name=['hip', 'hop','hip-hop']
new_genre_name='hiphop'
df['genre']=df['genre'].replace(drop_genre_name,new_genre_name)

#Проверка замены неявных дубликатов hiphop
list_one=df['genre'].sort_values().unique().tolist()
print(*filter(lambda x: x.startswith('h'), list_one))

handsup hard'n'heavy hardcore hardstyle hardtechno hiphop historisch holiday horror house hymn


### Выводы
Входе предобработки:
- Были обнаружены нарушения в стиле заголовков. Исправлено: наименования заголовков приведены к единому стилю.
- Были выявлены, пропуски в столбцах  `track`,  `artist` и  `genre`. Произведена замена пропусков явными обозначениями `unknown`
- Удалены явные и неявные дубликаты.

# Исследовательский анализ данных <a id='analysis'></a>

#### Проверка первой гипотезы
Первая гипотеза утверждает, что `активность пользователей в Москве и Санкт-Петербурге зависит от дня недели и для каждого города это проявляется по-разному`.
Для проверки гипотезы посчитаем прослушивания для заданного дня и запишем результаты расчетов в таблицу:


In [15]:
def number_tracks(day, city):
    track_list=df[df['day']==day]['user_id']
    track_list=track_list[df['city']==city]
    track_list_count=track_list.count()             
    return track_list_count

data = [
    ['Moscow',number_tracks('Monday', 'Moscow'),number_tracks('Wednesday', 'Moscow'),number_tracks('Friday', 'Moscow')],
    ['Saint-Petersburg',number_tracks('Monday', 'Saint-Petersburg'),number_tracks('Wednesday', 'Saint-Petersburg'),number_tracks('Friday', 'Saint-Petersburg')],
  ]
columns=['city', 'monday', 'wednesday', 'friday']
table_two= pd.DataFrame(data=data,columns=columns) 
print(table_two)

               city  monday  wednesday  friday
0            Moscow   15740      11056   15945
1  Saint-Petersburg    5614       7003    5895


### Выводы
Из таблицы видно, что
- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы, активность пользователей в Москве и Санкт-Петербурге действительно проявляется по-разному и зависит от дня недели.

#### Проверка второй гипотезы
Согласно второй гипотезе `утром в понедельник и в пятницу вечером в Москве преобладают одни жанры музыки, а в Петербурге — другие`. Для подтверждения или опровержения данного утверждения выведем количество прослушиваний каждого жанра в утренние и вечерние часы понедельника и пятницы для двух городов:


In [16]:
moscow_general=df[df['city']=='Moscow']
spb_general=df[df['city']=='Saint-Petersburg']

def genre_weekday(df, day, time1, time2):
    genre_df=df[df['day']==day]
    genre_df=genre_df[genre_df['time']>time1]
    genre_df=genre_df[genre_df['time']<time2]
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)
    return genre_df_sorted[:10]

morning=['07:00', '11:00']
day=['07:00', '11:00']
evening=['17:00', '23:00']
print('Прослушивание музыки в Москве',
             'Понедельник 07:00-11:00',genre_weekday(moscow_general, 'Monday', morning[0], morning[1]),
              'Пятница 17:00-23:00',genre_weekday(moscow_general, 'Friday', evening[0],evening[1]),
'Прослушивание музыки в Санкт-Петербурге',
      'Понедельник 07:00-11:00',genre_weekday(spb_general, 'Monday', morning[0], morning[1]),
      'Пятница 17:00-23:00',genre_weekday(spb_general, 'Friday', evening[0],evening[1]),sep='\n'*2)

Прослушивание музыки в Москве

Понедельник 07:00-11:00

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

Пятница 17:00-23:00

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

Прослушивание музыки в Санкт-Петербурге

Понедельник 07:00-11:00

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

Пятница 17:00-23:00

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap   

### Выводы
* В Санкт-Петербурге и Москве в понедельник утром и пятницу вечером слушают похожую музыку.
* Наибольшее количество пользователей (более 10%) слушают `популярную музыку`,`танцевальную`, `электронику` и `рок`. 
* В топ-10 жанров по Санкт-Петербургу вошли также `джаз` и `классика`, а по Москве -  `world`.
* В Москве пропущенных значений оказалось так много, что значение `unknown` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают музыку жанра world, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

#### Проверка третьей гипотезы
`Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.`
Для проверки данной гипотезы посчитаем количество прослушивания треков каждого жанра и выведем топ-10 жанров для Москвы и Санкт-Петербурга

In [17]:
moscow_genres=moscow_general.groupby('genre')['genre'].count()
moscow_genres=moscow_genres.sort_values(ascending=False)


spb_genres=spb_general.groupby('genre')['genre'].count()
spb_genres=spb_genres.sort_values(ascending=False)

print('Москва',moscow_genres[:10],
      'Санкт-Петербург',spb_genres[:10],sep='\n'*2)

Москва

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Санкт-Петербург

genre
pop            2431
dance          1932
rock           1879
electronic     1737
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64


### Выводы
Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге.

## Итоги исследования <a id='results'></a>

Мы проверили три гипотезы и установили:

 **1** День недели по-разному влияет на активность пользователей в Москве и Петербурге.

   Первая гипотеза полностью подтвердилась.

 **2** Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

 Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

 **3** Во вкусах пользователей Москвы и Петербурга больше общего, чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

   Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.